In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wiki-data2/all.csv
/kaggle/input/wiki-data2/train.csv
/kaggle/input/wiki-data2/test.csv


In [2]:
!pip install nltk stop-words

In [3]:
import torch
import torch.nn as nn
import re

In [4]:
df_train = pd.read_csv('/kaggle/input/wiki-data2/train.csv')
df_test = pd.read_csv('/kaggle/input/wiki-data2/test.csv')

In [5]:
df_train

,Unnamed: 0,title,content,label
0,0,Melocalamus scandens,Melocalamus scandens là một loài thực vật có h...,0
1,1,Paleauletobius silenus,Paleauletobius silenus là một loài bọ cánh cứn...,0
2,2,Zoodes hirsutus,Zoodes hirsutus là một loài bọ cánh cứng trong...,0
3,3,Quốc kỳ Argentina,Quốc kỳ Argentina ( tiếng Tây Ban Nha : Bander...,3
4,4,Dundahera,Dundahera là một thị trấn thống kê ( census to...,1
...,...,...,...,...
1720,2090,Volodymyr Oleksandrovych Zelensky,Volodymyr Oleksandrovych Zelensky ( tiếng Ukra...,3
1721,2091,Barack Obama,Barack Hussein Obama II ( / b ə ˈ r ɑː k h uː ...,3
1722,2092,Hillary Clinton,Hillary Diane Rodham Clinton ( phát âm tiếng A...,3
1723,2093,The New York Times,The New York Times (tên tiếng Việt: Thời báo N...,3


In [6]:
df_train['label'].value_counts()

label
0    1115
1     365
3     139
2      78
4      28
Name: count, dtype: int64

In [7]:
num_samples_to_remove = 600
indices_to_remove = df_train[df_train['label'] == 0].sample(n=num_samples_to_remove, random_state=42).index
df_filtered = df_train.drop(indices_to_remove)

In [8]:
df_filtered['label'].value_counts()

label
0    515
1    365
3    139
2     78
4     28
Name: count, dtype: int64

In [9]:
df_filtered['content']

1       Paleauletobius silenus là một loài bọ cánh cứn...
3       Quốc kỳ Argentina ( tiếng Tây Ban Nha : Bander...
4       Dundahera là một thị trấn thống kê ( census to...
5       Nạn khan hiếm nhu yếu phẩm tại Venezuela ( tiế...
7       Archidendron royenii là một loài thực vật có h...
                              ...                        
1720    Volodymyr Oleksandrovych Zelensky ( tiếng Ukra...
1721    Barack Hussein Obama II ( / b ə ˈ r ɑː k h uː ...
1722    Hillary Diane Rodham Clinton ( phát âm tiếng A...
1723    The New York Times (tên tiếng Việt: Thời báo N...
1724    Giải Pulitzer là một giải thưởng của Mỹ , trao...
Name: content, Length: 1125, dtype: object

In [10]:
a  = re.sub(r'\[\d+\]', '',df_filtered['content'].iloc[0])
a

'Paleauletobius silenus là một loài bọ cánh cứng trong họ Rhynchitidae . Loài này được Heer miêu tả khoa học năm 1847. '

In [11]:
df_filtered[df_filtered['content'].apply(lambda x: isinstance(x, str))]

,Unnamed: 0,title,content,label
1,1,Paleauletobius silenus,Paleauletobius silenus là một loài bọ cánh cứn...,0
3,3,Quốc kỳ Argentina,Quốc kỳ Argentina ( tiếng Tây Ban Nha : Bander...,3
4,4,Dundahera,Dundahera là một thị trấn thống kê ( census to...,1
5,5,Nạn khan hiếm nhu yếu phẩm tại Venezuela,Nạn khan hiếm nhu yếu phẩm tại Venezuela ( tiế...,1
7,7,Archidendron royenii,Archidendron royenii là một loài thực vật có h...,0
...,...,...,...,...
1720,2090,Volodymyr Oleksandrovych Zelensky,Volodymyr Oleksandrovych Zelensky ( tiếng Ukra...,3
1721,2091,Barack Obama,Barack Hussein Obama II ( / b ə ˈ r ɑː k h uː ...,3
1722,2092,Hillary Clinton,Hillary Diane Rodham Clinton ( phát âm tiếng A...,3
1723,2093,The New York Times,The New York Times (tên tiếng Việt: Thời báo N...,3


In [12]:
import re
import nltk
from stop_words import get_stop_words

nltk.download('stopwords')
stop_words = get_stop_words('vi')

def preprocess_text(text):
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\([^()]*\)', '', text)
#     words = text.split()
#     filtered_words = [word for word in words if word.lower() not in stop_words]
#     filtered_text = ' '.join(filtered_words)
    return text


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_filtered['content'] = df_filtered['content'].apply(preprocess_text)
df_test['content'] = df_test['content'].apply(preprocess_text)

In [14]:
df_filtered = df_filtered.sample(frac=1).reset_index(drop=True)
df_filtered.head()

,Unnamed: 0,title,content,label
0,1460,Jerzy Dudek,Jerzy Henryk Dudek là một cầu thủ bóng đá chu...,1
1,1983,Vật liệu composite,"Vật liệu composite , còn gọi là Vật liệu tổ hợ...",2
2,1759,Eleutherodactylus guentheri,Eleutherodactylus guentheri là một loài ếch tr...,0
3,800,"Xã Winsted, Quận McLeod, Minnesota","Xã Winsted là một xã thuộc quận McLeod , tiểu...",1
4,440,Hampea micrantha,Hampea micrantha là một loài thực vật có hoa t...,0


In [15]:
df_filtered

,Unnamed: 0,title,content,label
0,1460,Jerzy Dudek,Jerzy Henryk Dudek là một cầu thủ bóng đá chu...,1
1,1983,Vật liệu composite,"Vật liệu composite , còn gọi là Vật liệu tổ hợ...",2
2,1759,Eleutherodactylus guentheri,Eleutherodactylus guentheri là một loài ếch tr...,0
3,800,"Xã Winsted, Quận McLeod, Minnesota","Xã Winsted là một xã thuộc quận McLeod , tiểu...",1
4,440,Hampea micrantha,Hampea micrantha là một loài thực vật có hoa t...,0
...,...,...,...,...
1120,2044,Tất lưới,"Trong ngành may mặc, tất lưới là một loại tất...",3
1121,892,Phật giáo Nam truyền,"Phật giáo Nam truyền hay Phật giáo Nam tông , ...",3
1122,1091,"Abbottstown, Pennsylvania","Abbottstown là một thị trấn thuộc quận Adams ,...",1
1123,1637,Yên Thế (phường),"Yên Thế là một phường thuộc thành phố Pleiku ,...",1


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(df_filtered['content'])
X_test = tfidf_vectorizer.transform(df_test['content'])

In [18]:
y_train = df_filtered['label']
y_test = df_test['label']

In [19]:
# define model
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'KNN': KNeighborsClassifier()
}

In [20]:
# define params
params = {
    'SVM': {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['linear', 'rbf', 'poly']},
    'Random Forest': {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]},
    'Gradient Boosting': {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01, 0.001]},
    'KNN': {'n_neighbors': [3, 5, 7]}
}

In [21]:
# Thực hiện Grid Search cho từng mô hình
for model_name, model in models.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(model, params[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters found for {model_name}:")
    print(grid_search.best_params_)
    
    # Đánh giá trên tập test
    y_pred = grid_search.predict(X_test)
    print(f"Classification report for {model_name}:")
    print(classification_report(y_test, y_pred))

Training SVM...
Best parameters found for SVM:
{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Classification report for SVM:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       120
           1       0.72      0.86      0.79       100
           2       0.80      0.80      0.80        50
           3       0.80      0.74      0.77        80
           4       1.00      0.05      0.10        20

    accuracy                           0.82       370
   macro avg       0.85      0.68      0.68       370
weighted avg       0.83      0.82      0.80       370

Training Random Forest...
Best parameters found for Random Forest:
{'max_depth': None, 'n_estimators': 300}
Classification report for Random Forest:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       120
           1       0.59      0.86      0.70       100
           2       0.81      0.50      0.62        50
           3       0.68   

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
